In [1]:
import nmap
import ipaddress
import re
from typing import List, Dict, Any


In [11]:
scan_results = []
target_network="192.168.1.31/28"
nm=nmap.PortScanner()
try:
    nm.scan(hosts=target_network, arguments='-sn -sV')
    for host in nm.all_hosts():
        host_info = {
            'ip': host,
            'hostname': nm[host].hostname() or 'Unknown',
            'status': nm[host]['status']['state'],
            'open_ports': []
        }
        for proto in nm[host].all_protocols():
            ports = nm[host][proto].keys()
            for port in ports:
                port_details = {
                    'port': port,
                    'state': nm[host][proto][port]['state'],
                    'service': nm[host][proto][port]['name']
                }
                host_info['open_ports'].append(port_details)
        scan_results.append(host_info)
    print(scan_results)
except Exception as e:
    print(e)

[{'ip': '192.168.1.16', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.17', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.18', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.19', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.20', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.21', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.22', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.23', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.24', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.25', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.26', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.27', 'hostname': 'Unknown', 'status': 'up', 'open_ports': []}, {'ip': '192.168.1.28', 'hos

In [ ]:
from sqlite3 import connect, Connection, Cursor
db_path='security_monitor.db'
with connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS network_scans (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            ip_address TEXT,
            hostname TEXT,
            open_ports TEXT,
            scan_timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS security_events (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            event_type TEXT,
            severity TEXT,
            details TEXT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()

In [ ]:
from services.utils import logger,threat_signatures
import aiofiles

log_file_path = ""
threat_level = "medium"

async def analyze_logs(self) -> List[Dict[str, str]]:

    logger.info(f"Analyzing log file: {log_file_path}")
    detected_threats = []
    
    try:
        async with aiofiles.open(log_file_path, mode='r') as log_file:
            content = await log_file.read()
            
            for signature in threat_signatures.get(threat_level, []):
                if signature.lower() in content.lower():
                    detected_threats.append({
                        'signature': signature,
                        'risk_level': self.threat_level.capitalize(),
                        'details': f'Potential threat detected: {signature}'
                    })
        
        return detected_threats
    
    except FileNotFoundError:
        logger.error(f"Log file not found: {self.log_file_path}")
        return []
    except Exception as e:
        logger.error(f"Log analysis error: {e}")
        return []
